# Dissertation Data Analysis Notebook 

**This notebook contains the workflow for my dissertation on the control of folded geology on drainage network patterns.**

* Author: Rutger Vervoordeldonk
* Last updated: 07/02/2024

This notebook provides the full workflow of my dissertation data analysis. It uses:
1. `HydroBasins` datasets to select river basins
2. `LSDTopoTools` to extract river networks from Copernicus 30m resolution Digital Elevation Models (accessed through the `OpenTopography` API).
3. `rivernetworkpy` (`rnp`): a (local) python package that contains the data analysis scripts for this notebook
4. `WVGES Geological Data` to  compare orientations of network segments to geological structures
5. `HydroLakes` datatasets to remove sections of river networks where there are lakes

The data analysis can be subdivided into roughly two parts:
* Part 1: The Segment Orientation Analysis with Bedding Data (SOA-BED). 
* Part 2: The Segment Orientation Analysis w/o Bedding Data (SOA) and the Junction Geometry Analysis (JGA) 
The analysis is divided this way on the basis that Part 1 requires networks to be extracted slightly differently than in Part 2. This will become clear in the individual Parts. 

The final output of this analysis are
* SOA/SOA-BED plots: Half-dial plots showing segment orientation distributions. This plot uses data generated both in Part 1 and Part 2. In addition, also a Test for Uniformity, Unimodality, Bimodality of Segment orientations will be performed
* JGA plots: Junction Geometry Kuipers Boxplots, Junction Geometry CDF plots and $A_R$ CDF plots. Draws only upon data of Part 2

## Setup file directory

(Manually) setup a directory with the following structure:

- **basins**: _an empty subdirectory that will be used to store the data of individual basins that are downloaded and analysed. Includes sub-subdirectories **PART1** and **PART2**_
- **hydrobasins**: _a subdirectory with shapefiles of the levels 5, 6 and 7 of the Central and North America HydroBasins (downloadable from their website). Also useful to download HydroRIVERS of the same area to this folder; HydroRIVERS is not necessary for the analysis but can come in handy when picking HydroBASINS for analysis._
- **hydrolakes**: _a subdirectory with shapefiles of hydrolakes of North America. Also downloadable from their website. You should then use QGIS to remove all lakes outside the area you are interested in to save a lot of computation time later._
- **rivernetworkpy-base**: _a subdirectory containing the `rivernetworkpy` package_
- **wvbed**: _a subdirectory containing the West Virginia bedding measurements shapefile AND the shapefile of corresponding quadrangles_
- **vrpapoutlines** _a subdirectory containing two shapefiles, one for the Valley and Ridge and one for the Appalachian Plateau, describing the outline of the two physiographic provinces. I derived those from Fennemans map of provinces downloadable from the USGS, as described in my dissertation. I can email them to you if you want them. 
- my_OT_api_key: _a txt file only containing your API key from Opentopography.org. To get an API key you have to make an account there._
- a QGIS (or ArcGIS) file: _useful to open and inspect the WV_BED shapefile and HydroBasins layers, plus any geospatial data generated during this workflow_
- this notebook

Later, this notebook will generate additional subdirectories for result data

## Setup `rivernetworkpy`

To setup `rivernetworkpy`:
1. make sure that you are in the right conda environment 
2. using the terminal, cd yourself to the rivernetworkpy-base directory: `cd your-path/rivernetworkpy-base`
3. develop the local python package: `python3 setup.py develop`

Luckily, you only have to do this once! If you were to change something to `rivernetworkpy`, just restart the kernel of this notebook and re-import the package - there's no need to re-develop the package.

## Import python packages

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw  # lsdmw is a python driver for LSDTopoTools command line tools!
import pandas as pd # to read csv's 
import os # needed later to create subdirectories in the basins folder
import rivernetworkpy as rnp # rnp contains all data-analysis scripts for this workflow

# Part 1

## Select HydroBasins for analysis

Select the HydroBasins for analaysis by entering HYBAS_IDs and intuitive nicknames (only letters, numbers and underscores!) in the dictionary below. HYBAS_IDs can be inspected using QGIS. 

By default, the dictionary contains the hydrobasins that I used for the analysis. 

In [ ]:
part1_hybas_ids = {
    7070547240 : "WV_VR_West",
    7050041400 : "WV_VR_North",
    7050569090 : "WV_VR_South"
}

## Setup folders for selected HydroBasins and download DEMs

For a given HydroBasin, the `setup_directory_with_DEM` function will setup a folder in the subdirectory **basins** and download a DEM using `LSDTopoTools`' OpenTopography Scraper. 

Note that, if a folder already exists with the same nickname, no new folder will be set up and no DEM will be downloaded for this hydrobasin.

In [ ]:
for hybas_id in part1_hybas_ids:
    print("------------NOW STARTING WITH: "+part1_hybas_ids[hybas_id]+"------------------")
    rnp.setup_directory_with_DEM(hybas_id, part1_hybas_ids[hybas_id], part = "PART1")

## Perform flow routing and calculate segment bearings for each basin using `LSDTopoTools`
Note that the function `run_lsdtt_for_basin` is defined here and not in `rivernetworkpy` to make it a bit easier to adjust lsdtt parameters.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
    
def run_lsdtt_for_basin(nickname, part):
    Dataset_prefix = nickname
    source_name = "COP30"
    DataDirectory = "./Basins/"+part+"/"+Dataset_prefix+"/"
    
    lsdtt_parameters = {"print_segment_bearings_and_gradients_to_csv" : "true",
                        "SA_vertical_interval" : "10",
                        "threshold_contributing_pixels" : "500", 
                        "print_chi_data_maps":"true",
                        "write_hillshade" : "true",
                        "convert_csv_to_geojson":"false",
                        "only_take_largest_basin":"true",
                        "maximum_basin_size_pixels":"10000000000000",
                        "print_basin_raster":"true"}

    r_prefix = Dataset_prefix+"_"+source_name+"_UTM_clipped"
    w_prefix = Dataset_prefix+"_"+source_name+"_UTM"

    lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                     read_prefix = r_prefix,
                                     write_prefix= w_prefix,
                                     read_path = DataDirectory,
                                     write_path = DataDirectory,
                                     parameter_dictionary=lsdtt_parameters)
    lsdtt_drive.print_parameters()
    lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
for hybas_id in part1_hybas_ids:
    print("------------NOW STARTING WITH: "+part1_hybas_ids[hybas_id]+"------------------")
    run_lsdtt_for_basin(part1_hybas_ids[hybas_id], part = "PART1")

There now should be a `chi_data_map.csv` file that containing the channel network in every folder, as well as `csv` files containing the segments and their bearings (called `_segment_bearings.csv`).

The segment bearings csv contains all bearings within the DEM (so also those outside the basin we are interested in (the largest basin), on the edge of the DEM). 

This can be fixed through some merging of the segment bearings csv with the chi data map (the chi data map is only of the largest basin), based on lat/lons of the network's nodes:

In [ ]:
for hybas_id in part1_hybas_ids:
    print("------------NOW STARTING WITH: "+part1_hybas_ids[hybas_id]+"------------------")
    rnp.merge_segment_bearings_with_chi_data_map_for_basin(part1_hybas_ids[hybas_id], part = "PART1")

## Find bedding measurements along channels

Now, we are going to run for each basin a function that finds all bedding measurements within a 1km radius of all along-stream segments of a network that are inside the VRP Quadrangles with bedding measurements and outside lakes. 

In order to generate this dataset, the function sequentially:
* Obtains along-stream segments as line data from the chi data map using junctions contained in the segment_bearings_dataset. 
* writes out a GeoJSON (shape) file containing the lines of the river network. This should be manually inspected in QGIS to verify that all the points were correctly converted into lines.
* discards of all lines that are not fully contained within the `WV_QUADS_VRP.shp` polygon
* also discards of all lines that lie (partially or fully) inside a `HydroLAKES` polygon. **It is advisable to restrict the spatial extent of the HydroLAKES dataset to NE USA before running this code, otherwise it will take forever!**
* writes out another GeoJSON file with the lines, such that you can check that the previous two steps went correctly
* buffers the remaining lines by a radius of 1000m
* identifies for each buffered line segment all bedding measuruments (`WV_BED_VRP.shp`) that lie within the buffer (**this step takes a long time!**)
* writes out, to each PART1 basin folder, a csv ending in `_segment_bearings_bedding_measurements.csv`
* writes out, to the PART1 basin folder, as csv containing all unique bedding measurements found within all the buffers. This is to, later, calculate means of strike and dip and also the total number of linked bedding measurements. Note that you would not be able to get unique values from just loping over all the bedding measurements in `_segment_bearings_bedding_measurements.csv` as that would lead to double values (1 bed measurement can be connected to multiple streams)!
  
The result of this function is a csv file ending in `_bedding_measurements.csv`. In the next step we will match the bedding measurements to the `_segment_bearings.csv` by matching the junction numbers. 


In [ ]:
for hybas_id in part1_hybas_ids:
    print("------------NOW STARTING WITH: "+part1_hybas_ids[hybas_id]+"------------------")
    rnp.add_bedding_measurements_to_segments(part1_hybas_ids[hybas_id])

The next function will link add the bedding measurements to the `_segment_bearings.csv`, by merging based on junction numbers. 

The following columns are added to the `_segment_bearings.csv`:
* azimuths, dips, dip_directions: the 'raw' data from the WVGES bedding measurements (directed data)
* strikes, dip_orientations: the strikes and dip orientations derived from the 'raw' data, reduced to the interval [0, 180) (oriented data)
* n_strike, n_dip_orientation: number of strike and dip orientation measurements (should be the same)
* mean_strike, mean_dip_orientation: means of strike and dip orientation (used to calculate difference with segment orientation)
* circ_var_strike, circ_var_dip_orientations: circular variance of strikes and dip orientations (not used but just there for fun)
* segment_orientation: the segment bearing with directionality removed, such that all data on interval [0,180) (oriented data)
* difference_segment_orientation_and_mean_strike (on interval [-90, 90])
* difference_segment_orientation_and_mean_dip_orientation (on interval [-90, 90])

The result is saved as a csv ending in `_segment_bearings_bedding_stats.csv`

In [ ]:
for hybas_id in part1_hybas_ids:
    print("------------NOW STARTING WITH: "+part1_hybas_ids[hybas_id]+"------------------")
    rnp.add_bedding_measurements_to_segment_bearings(part1_hybas_ids[hybas_id])

Next up, we are going to merge the `segment_bearings_bedding_stats.csv` and the `_unique_bedding_measurements.csv` files of the individual basins and save these in the `PART1` directory. 

In [ ]:
rnp.merge_segment_bearings_bedding_stats(part1_hybas_ids)
rnp.merge_unique_bedding_measurements(part1_hybas_ids)

We now have now prepared the dataset of river networks connected to bedding measurements. We will analyse this data later. We are first going to extract basins for Part 2. 

# Part 2

## Setup directories in PART2 and Download DEMs 
The procedure here is slightly different to Part 1. In Part 1 we wanted to find _all channels_ fully within the extent of VRP bedding measurements, thus also streams whose basins lie partially outside this extent.

In Part 2, we want to find _as many basins of similar size_ that lie fully wihtin the extent of either the VRP or the AP.

It would be easiest to download one DEM with the extent of the VRP polygon and one with the extent of the AP polygon, and then use LSDTopoTools `find_basins` function to find similarly sized basins. 

These DEMs would, however, be huge (>1 GB) and therefore require lots of memory when flow routing. The VRP and AP polygons must therefore be sensibly subdivided (i.e. along water divides) first. 

This will be done as follows:
* First make a dictionary where the key represents a VRP or AP subdivision, then the values are Hydrobasin IDs within this subdivision.
* Then, for each subdivision, collapse the Hydrobasin outlines into one geometry, buffer this by 1000m (buffer is necessary because Hydrobasins water divides - derived from 90m resolution DEMs - are expected to be slighlty off from those derived from 30m using LSDTopoTools), and find the intersection of this buffered polygon and the VRP or AP outline.
* The result is polygons that represent sensible subdivisions of the VRP and AP.

Later, we can download DEMS according to these polygons and perfrom flow routing, and later merge all the basins found together into one big CSV containing all data of the basins!

In [ ]:
subdivisions = {
    "VRP_SOUTH": [7060597760],
    "VRP_MID_S": [7070597680, 7070594640, 7070594650, 7070598130, 7070598140, 7070611080],
    "VRP_MID_N": [7050041400],
    "VRP_NORTH": [7050041050, 7060040360],
    "AP_NORTH": [7060529340, 7060519270],
    "AP_MID_N": [7060529460, 7060523200],
    "AP_MID_M": [7060555510, 7060555650, 7060558980, 7060558770, 7060568950],
    "AP_MID_S": [7060569090, 7060580820, 7060597680, 7060585550, 7060579690, 7060580930, 7060579680, 7060585700, 7070082250, 7070598400, 7060597760], 
    "AP_SOUTH": [7060571990, 7060571810, 7060562310, 7060572890]
}

In [ ]:
import rivernetworkpy as rnp
rnp.setup_directories_with_subdivision_outlines(subdivisions)

The above function set up file directories of the subdivisions and put a GeoJSON file of the outline in each. 

Checking in QGIS, you can see that all the outlines are correct except `AP_NORTH` and `AP_MID_N`. These subdivisions have a 'gap' running through the middle, which is due to the two regions of the AP not properly aligning on the small scale in the Physiographic Provinces dataset. It is very hard to remove the gap with code, so, before continuing, go to QGIS, load these two GeoJSONs and for each layer click 'Toggle Editing' and use the 'Vertex Tool' to remove the nodes that make up the gaps.

Once that is done, run the next function that for each subdivision downloads the right DEM. That will take a while (depending on your internet connection) so make yourself lunch in the meantime. Dont forget to close the GeoJSONs you just opened (and adjusted), because otherwise the next thing won't work.

In [ ]:
import rivernetworkpy as rnp
rnp.prepare_DEMS_for_subdivision_outlines(subdivisions)

## Use `LSDTopoTools` to perform flow routing, find basins and get segment bearings and junction angles
We first redefine the run_lsdtt_for_basin function to include "get junction angles" and "find basins", and remove "only_take_largest_basin".

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
    
def run_lsdtt_for_basin(nickname, part):
    Dataset_prefix = nickname
    source_name = "COP30"
    DataDirectory = "./Basins/"+part+"/"+Dataset_prefix+"/"
    
    lsdtt_parameters = {"print_segment_bearings_and_gradients_to_csv" : "true",
                        "print_junction_angles_to_csv_in_basins":"true",
                        "SA_vertical_interval" : "10",
                        "threshold_contributing_pixels" : "500", 
                        "print_chi_data_maps":"true",
                        "write_hillshade" : "true",
                        "convert_csv_to_geojson":"false",
                        "only_take_largest_basin":"false",
                        "find_basins":"true",
                        "maximum_basin_size_pixels":"10000000000000",
                        "print_basin_raster":"true"}

    r_prefix = Dataset_prefix+"_"+source_name+"_UTM_clipped"
    w_prefix = Dataset_prefix+"_"+source_name+"_UTM"

    lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                     read_prefix = r_prefix,
                                     write_prefix= w_prefix,
                                     read_path = DataDirectory,
                                     write_path = DataDirectory,
                                     parameter_dictionary=lsdtt_parameters)
    lsdtt_drive.print_parameters()
    lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
for subdivision in subdivisions:
    run_lsdtt_for_basin(subdivision, part = "PART2")

As in Part 1, we need to make sure the segment bearings are reduced to just those in the basins and not the entire DEM:

In [ ]:
for subdivision in subdivisions: 
    rnp.merge_segment_bearings_with_chi_data_map_for_basin(subdivision, part = "PART2")

Add the basin_key to the junction angles csv: 

In [ ]:
for subdivision in subdivisions:
    rnp.add_basin_key_to_junction_angles_csv(subdivision, part = "PART2")

## Further preparation of junction angle and segment bearings csv data for analysis and plotting

Looking at the basin rasters in QGIS, there is a lot of variation in basin size. 
I only want to include basins in the analysis that are at least of order 5:

In [ ]:
for subdivision in subdivisions: 
    rnp.remove_low_order_basins_from_segment_bearings_csv(subdivision, part = "PART2", threshold=5)
    rnp.remove_low_order_basins_from_junction_angles_csv(subdivision, part = "PART2", threshold=5)

The result is saved in csvs with 'filtered' in the name.

As in part 1, the measurements from river segments that are in lakes should be removed. This is done in the same way as in part 1; if any part of a segment is in a lake, then the segment will be removed from the segment bearings dataset, and so is the upstream junction from the junction angles dataset.

The resulting csv's end in `_filtered_nolakes.csv`. As before, two geojsons are produced (`segment_lines.geojson` and `segment_lines_nolakes.geojson`) so you can check if the lake filtering went as expected.

In [ ]:
for subdivision in subdivisions:
    rnp.remove_lake_segment_bearings_and_junction_angles(subdivision)

Now the segment bearings and junction angles csv's of the subdivisions are ready to use. Let's merge them into one segment bearings csv and one junction angles csv in the PART2 directory:

In [ ]:
import rivernetworkpy as rnp
rnp.create_all_segment_bearings_and_junction_angles_csvs(subdivisions)

Note that a new column "new_basin_key" is added. This is to ensure each basin remains to have a unique identifier. The first digit of the new basin key indicates the subdivision it originated from (the first subdivision in the subdivisions dict being 1), and the remaining digits indicate the original basin_key.

The two CSV's are now ready to be analysed.

# Data Analysis and Plotting

## Junction Angles
Want to plot the following things in one figure:
1. Cumulative Density Functions (CDF's) of the junction angles of each basin, of each angle (A, B1, B2) into one plot. AP CDF's should be coloured one colour, and VRP another colour.
2. A boxplot with for each angle (A, B1, B2) three boxes: the Kuiper's V-statistic (which is calculated between two CDF's) for the groups VRP-VRP, AP-AP and AP-VRP.
3. Cumulative Density Functions (CDF's) of the Area Ratios of each basin. AP and VRP CDF's should be coloured according to the colours in the previous two plots.


In [ ]:
import rivernetworkpy as rnp
rnp.junction_angles()

## Segment orientations

In [ ]:
import rivernetworkpy as rnp
rnp.segment_orientations()

## Halfdials

In [ ]:
import rivernetworkpy as rnp
rnp.halfdials(hspace=0)

# Extra code used to prepare other diagrams and stats used in dissertation

## Calculating total area of VRP bedding measurements for stats in Introduction chapter

In [ ]:
import geopandas as gpd

# Load the shapefile
file_path = './wvbed/WV_QUADS_VRP.shp'
gdf = gpd.read_file(file_path)

# Dissolve the polygons into a single geometry
dissolved_gdf = gdf.dissolve(aggfunc='sum') 

# Convert the geometry to EPSG:32617 - WGS 84 / UTM zone 17N for Cartesian area calculation
dissolved_gdf = dissolved_gdf.to_crs(epsg=32617)

# Calculate the Cartesian area in square kilometers
dissolved_gdf['cartesian_area_km2'] = dissolved_gdf['geometry'].area / 1e6  # Convert square meters to square kilometers

# Print the result
print("The resurveyed quads of west virginia have a total area of: " + str(dissolved_gdf['cartesian_area_km2'].tolist()[0]) + "km2")


## For methods diagram: get empirical cumulative probability function graphs for some VRP and AP basins, and get hypothesised distributions

Make folder to store these graphs

In [ ]:
import os 
if os.path.exists("./methodsfig") == False:
    os.mkdir("./methodsfig")

Get CDF of 1 emperical and 1 generated distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rivernetworkpy as rnp

# load data
sb_df = pd.read_csv("./basins/PART2/all_segment_bearings.csv")
    
# remove directionality from bearings, i.e. make orientations column on [0,180)
sb_df["segment_orientation"] = rnp.remove_directionality(sb_df["segment_bearing_whole_segment"].tolist())

# get orientations
orientation_distribution = sb_df[sb_df["new_basin_key"]==25]["segment_orientation"].to_numpy()

# get hypothesised distribution
hypothesised_distribution = rnp.generate_distribution(p1=1, peak=0, distribution="bimodal", s = 30, m = 90, n = 1000)

# put in list
distributions = [orientation_distribution, hypothesised_distribution]

# Create a common set of values for the x-axis
x_values = np.linspace(0, 180, 1000)

# create figure
plt.figure(figsize=(4,4))

# select colours
colours = ["darkorange", "cornflowerblue"]

for distribution, colour in zip(distributions, colours):
    # Calculate the empirical cumulative distribution function (ECDFs)
    ecdf = np.searchsorted(np.sort(distribution), x_values, side='right') / len(distribution)
    
    # Plot ecdf
    plt.step(x_values, ecdf, where='post', color = colour)

plt.xlim((0,180))
plt.xticks([0,45,90,135,180], ["N", "NE", "E", "SE", "S"])

plt.ylim((0,1))
plt.yticks([0,1])
plt.xlabel('Orientation')
plt.ylabel('Cumulative Probability')
plt.savefig("./methodsfig/ECDF_CDF.jpg", dpi=360)

# get V value (note this code is identical to the Calculate Kuipers V function in Rivernetworkpy BUT now also get D+ and D-
# Calculate the empirical cumulative distribution functions (ECDFs)
ecdf1 = np.searchsorted(np.sort(orientation_distribution), x_values, side='right') / len(orientation_distribution)
ecdf2 = np.searchsorted(np.sort(hypothesised_distribution), x_values, side='right') / len(hypothesised_distribution)

# Calculate the differences between ECDFs
differences = ecdf1 - ecdf2

# Calculate the maximum positive and maximum negative differences
max_positive_difference = np.max(np.clip(differences, 0, None))
max_negative_difference = np.abs(np.min(np.clip(differences, None, 0)))

# get location of D+ and D-
max_positive_index = round(np.argmax(differences)/1000*180)
max_negative_index = round(np.argmax(-differences)/1000*180)

# Calculate the combined value V
v = max_positive_difference + max_negative_difference
print("D+, D- and V are:", max_positive_difference,max_negative_difference,v)
print("D+ and D- are located at", max_positive_index, max_negative_index)


Now get empirical cdf's for 1 VRP and 1 AP basin:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rivernetworkpy as rnp

# load data
sb_df = pd.read_csv("./basins/PART2/all_segment_bearings.csv")
    
# remove directionality from bearings, i.e. make orientations column on [0,180)
sb_df["segment_orientation"] = rnp.remove_directionality(sb_df["segment_bearing_whole_segment"].tolist())

# select basins by new_basin_key
new_basin_keys = [25, 735]

# Create a common set of values for the x-axis
x_values = np.linspace(0, 180, 1000)

# create figure
plt.figure(figsize=(4,4))

for new_basin_key in new_basin_keys:
    # get orientations
    orientations = sb_df[sb_df["new_basin_key"]==new_basin_key]["segment_orientation"].to_numpy()

    # Calculate the empirical cumulative distribution function (ECDFs)
    ecdf = np.searchsorted(np.sort(orientations), x_values, side='right') / len(orientations)

    # get colour:
    color = "darkorange" if (int(str(new_basin_key)[0]) < 5) else "limegreen"
    
    # Plot ecdf
    plt.step(x_values, ecdf, where='post', color = color)

plt.xlim((0,180))
plt.xticks([0,45,90,135,180], ["N", "NE", "E", "SE", "S"])

plt.ylim((0,1))
plt.yticks([0,1])
plt.xlabel('Orientation')
plt.ylabel('Cumulative Probability')
plt.savefig("./methodsfig/VRP_AP_BASIN_ECDF.jpg", dpi=360)

## Code for second methods diagram, the distributions

In [ ]:
import rivernetworkpy as rnp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Set up plot
fig = plt.figure(figsize=(9,6))
gs_outer = gridspec.GridSpec(2,1)
gs_upper = gridspec.GridSpecFromSubplotSpec(1,3, subplot_spec=gs_outer[0]) # for the distributions of the fist type
gs_lower = gridspec.GridSpecFromSubplotSpec(1,3, subplot_spec=gs_outer[1]) # for the distributions of the second type

# Create a common set of values for the x-axis
x_values = np.linspace(0, 180, 1000)

# set linestyles and colours 
ls_list = ["--","--"]
colours = ["cornflowerblue", "grey"]

for gsnumber in [0,1,2]:
    # get p from gsnumber
    p1 = gsnumber/2

    # plot the distribution of the first type for this p
    ax_1 = plt.subplot(gs_upper[gsnumber])
    distributions_1 = [rnp.generate_distribution(p1=p1, peak=0, distribution="bimodal", s = 30, m = 90, n = 1000), rnp.generate_distribution(p1=p1, peak=45, distribution="bimodal", s = 30, m = 90, n = 1000)]
    for distribution, ls, colour in zip(distributions_1, ls_list, colours):
        # Calculate the cumulative distribution function 
        cdf = np.searchsorted(np.sort(distribution), x_values, side='right') / len(distribution)
        # Plot ecdf
        ax_1.step(x_values, cdf, where='post', color = colour, ls=ls)

    # plot the distribution of the second type for this p
    ax_2 = plt.subplot(gs_lower[gsnumber])
    distributions_2 = [rnp.generate_distribution(p1=p1, peak=0, distribution="skewed_bimodal", s = 30, m = 90, n = 1000), rnp.generate_distribution(p1=p1, peak=45, distribution="skewed_bimodal", s = 30, m = 90, n = 1000)]
    for distribution, ls, colour in zip(distributions_2, ls_list, colours):
        # Calculate the cumulative distribution function 
        cdf = np.searchsorted(np.sort(distribution), x_values, side='right') / len(distribution)
        # Plot ecdf
        ax_2.step(x_values, cdf, where='post', color = colour, ls=ls)

    # do plot layout
    for ax in [ax_1, ax_2]:
        ax.set_ylim((0,1))
        ax.set_xlim((0,180))
        ax.set_yticks([0,1])
        ax.set_xticks([0,45,90,135,180],["N","NE","E", "SE","S"])

plt.savefig("./methodsfig/distributions.jpg", dpi=360)